# 변수명 안내
- total_data : 수집된 키워드별 리뷰 데이터
- stopwords : 불용어 집합 변수
- threshold : 최소 빈도수 값 정의

In [ ]:
!pip install konlpy

In [1]:
import time
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('seaborn-white')

from konlpy.tag import Okt

## 데이터 로드

In [26]:
#csv
# total_data = pd.read_csv(
#     #'./ReviewDataSet/Lodgment_Integrated.csv', ###SNS에서 수집한 데이터 로드
#     encoding='CP949'
# )

#excel
total_data = pd.read_excel('gwangjuRest.xlsx')

#데이터 value의 수 확인
print(len(total_data))

1000


In [6]:
#10개 value 확인
total_data[:10]

,Unnamed: 0,title,link,description,bloggername,bloggerlink,postdate
0,0,"국내 여행 갈 때 필수 아이템 된 지역화폐카드, 최대 10% 캐시백...",https://blog.naver.com/goldentreeconsulting?Re...,제주도 등 국내 <b>여행지</b>를 찾는 관광객이 많아지고 있다. 제주도관광협회에...,싱가포르 법인설립 전문회사,https://blog.naver.com/goldentreeconsulting,20210603
1,1,"[정치포커스] <b>광주광역시</b> 광산구, ‘<b>광주</b> 유니버시티...",https://blog.naver.com/jebofocus?Redirect=Log&...,"싶은 <b>여행지</b>, <b>광주</b>를 만드는 첫 출발점”이라며 &quot;...",정치포커스,https://blog.naver.com/jebofocus,20210602
2,2,[이화자수] 의상에 자수놓기/ 조끼 자수/ 의상리폼하기...,https://blog.naver.com/2236610?Redirect=Log&lo...,다양한 <b>여행지</b>를 아무런 제약 없이 핸드폰 하나로 자유롭게 볼 수 있기에...,꿈꾸는 자수공장:),https://blog.naver.com/2236610,20210602
3,3,첫글 _ 백문 백답,https://blog.naver.com/bc91102?Redirect=Log&lo...,<b>광주광역시</b> 7. 지금 살고있는 동네는? 띵동 비밀 8. 남들이 모르는 ...,ᴍʏ sᴘᴀᴄᴇ,https://blog.naver.com/bc91102,20210602
4,4,1박2일/2박3일 국내여행 패키지 추천 상품 (일정/가격/예약),https://blog.naver.com/hnltour?Redirect=Log&lo...,<b>광주광역시</b>와 <b>광주</b>관광재단에서 지원하는 상품으로 매우 저렴한...,봄‧여름‧가을‧겨울 행복충전 하늘투어,https://blog.naver.com/hnltour,20210601
5,5,6월 <b>여행지</b> 추천 양림동 펭귄마을에서 즐길 수 있는 BEST 3,https://blog.naver.com/kyoung700530?Redirect=L...,신청대상 <b>광주광역시</b> 소재 사업자를 보유하고 있는 예술인 등 신청방법 양...,블루 바람돌이,https://blog.naver.com/kyoung700530,20210601
6,6,6월 <b>여행지</b> 추천 양림동 펭귄마을에서 즐길 수 있는 BEST 3,https://blog.naver.com/o-hands?Redirect=Log&lo...,양림동 펭귄마을의 매력 BEST 3 [6월 <b>여행지</b> 추천] 거리 곳곳에 ...,오핸즈_광주수제공방육성사업,https://blog.naver.com/o-hands,20210601
7,7,전라도 <b>여행지</b> 추천 전남 장성 황룡강 생태공원 꽃구경 맘껏,https://blog.naver.com/gebera12?Redirect=Log&l...,29 전남 지역 코로나19 확산세로 1달을 주말에 나가지도 않고 집콕만 하던 아델레...,아델레 Story,https://blog.naver.com/gebera12,20210601
8,8,<b>광주광역시</b> 근교 산책하기 좋은 화순 세량제 세량지 / 화순군...,https://blog.naver.com/kkjjgg1914?Redirect=Log...,왼쪽에 보이는 건물이 아까 그 화장실이에요 세량제는 저 방향입니다~~ 지도는 참고하...,KnB홈케어 (Kitchen & Bathroom Homecare),https://blog.naver.com/kkjjgg1914,20210529
9,9,100문 100답,https://blog.naver.com/coinseo?Redirect=Log&lo...,"나는 그 책을 읽으면서 엄청 치를 떨었는데 그게 실화라는 것에 한 번 놀랬고, <b...",815,https://blog.naver.com/coinseo,20210529


## 불필요 컬럼 제거

In [27]:
#컬럼 확인
total_data.columns

Index(['Unnamed: 0', 'title', 'link', 'description', 'bloggername',
       'bloggerlink', 'postdate'],
      dtype='object')

In [28]:
#리뷰내용컬럼(description) 빼고 제거
total_data = total_data.drop(['Unnamed: 0', 'title', 'link', 'bloggername', 'bloggerlink', 'postdate'], axis=1)

In [29]:
#컬럼명 변경
total_data.columns = ['document']

#label컬럼 추가
total_data["label"] = 0

## 중복 및 결측치 처리
- 데이터 개수 확인
- 데이터에 중복이 존재한다면 이를 제거

In [30]:
#유니크한 value의 개수 확인
print(total_data['document'].nunique())

# 중복 제거
total_data.drop_duplicates(subset=['document'], inplace=True)

1000


In [31]:
#결측치 개수 확인
print(total_data.isnull().sum())

# 결측치 제거
total_data = total_data.dropna(how='any')

document    0
label       0
dtype: int64


## 데이터 정제
- 데이터에서 한글과 공백을 제외하고 모두 제거

In [32]:
# replace 에 포함된 나머지 문자들 제거
total_data['document'] = total_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")

total_data[:10]

,document,label
0,제주도 등 국내 여행지를 찾는 관광객이 많아지고 있다 제주도관광협회에 따르면 월 한...,0
1,싶은 여행지 광주를 만드는 첫 출발점이라며 로컬크루가 역량을 맘껏 펼칠 수 있도록 ...,0
2,다양한 여행지를 아무런 제약 없이 핸드폰 하나로 자유롭게 볼 수 있기에 유튜브 시장...,0
3,광주광역시 지금 살고있는 동네는 띵동 비밀 남들이 모르는 나 관종이다 그리고 상...,0
4,광주광역시와 광주관광재단에서 지원하는 상품으로 매우 저렴한 가격에 다녀오실 수 있는...,0
5,신청대상 광주광역시 소재 사업자를 보유하고 있는 예술인 등 신청방법 양림동펭귄마을공...,0
6,양림동 펭귄마을의 매력 월 여행지 추천 거리 곳곳에 초록이 짙어지는 월의 막바지...,0
7,전남 지역 코로나 확산세로 달을 주말에 나가지도 않고 집콕만 하던 아델레네 가족 ...,0
8,왼쪽에 보이는 건물이 아까 그 화장실이에요 세량제는 저 방향입니다 지도는 참고하세요...,0
9,나는 그 책을 읽으면서 엄청 치를 떨었는데 그게 실화라는 것에 한 번 놀랬고 광주광...,0


In [7]:
# 빈 값이 얼마나 되는지 확인
total_data['document'].replace('', np.nan, inplace=True)
print(len(total_data))
print(total_data.isnull().sum())

36147
document    16
label        0
dtype: int64


In [8]:
# 빈 공간 제거
total_data = total_data.dropna(how='any')
print(len(total_data))

36131


In [ ]:
#저장
total_data.to_csv('Lodgment_Integrated.csv', encoding = 'CP949')